# Collaborative Filtering Recommender System based on Cosin Similarity

### Purpose
To get a working cosin similarity model based off of collaborative filtering

### Methodology
This notebook assumes that the model will receive a pre-processed dataset of user-item interactions. For simplification purposes, it uses the [small movielens dataset](https://surprise.readthedocs.io/en/stable/dataset.html)

### Author Information
Nishant Aswani (@niniack)


### Setup (Imports)

In [1]:
# Data manipulation
import pandas as pd
import numpy as np
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, Predictor, als, basic, user_knn
from lenskit import topn
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

# Dataset
from lenskit.datasets import ML100K
movielens = ML100K('../ml-100k')

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Visualizations
import plotly.graph_objs as go
from pprintpp import pprint as pp

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

### Downloading ML100K Dataset

In [2]:
# %%!
# wget -q -O ml-100k.zip http://files.grouplens.org/datasets/movielens/ml-100k.zip

## This unzip method may not work!
# unzip -f ml-100k.zip

### Data Exploration

The lenskit ML100K dataset provides the following: movies, ratings, users

In [3]:
ratings = movielens.ratings
ratings.head()

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [4]:
len(ratings)

100000

In [5]:
users = movielens.users
users.head()

,age,gender,occupation,zip
user,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [6]:
len(users)

943

In [7]:
movies = movielens.movies
movies.head()

,title,release,vidrelease,imdb,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
item,,,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [8]:
len(movies)

1682

### Testing The "Most Popular Item Recommendation" System

The popular recommender calculates a score for each item in the ratings matrix. When provided a user to recommend items to, the model returns the top scoring n items that the given user has not previously rated.

In [9]:
# Initializing and "training" the popular recommender
algo_popular = basic.Popular()
algo_popular.fit(ratings)

In [10]:
# Recommend the top 10 most popular items for UserID 20
algo_popular.recommend(20, 10)

,item,score
0,258,509.0
1,100,508.0
2,294,485.0
3,286,481.0
4,300,431.0
5,127,413.0
6,56,394.0
7,7,392.0
8,237,384.0
9,117,378.0


In [11]:
pop = ratings.groupby('item').user.count()
pop.sort_values(ascending=False)

item
50      583
258     509
100     508
181     507
294     485
       ... 
1576      1
1577      1
1348      1
1579      1
1682      1
Name: user, Length: 1682, dtype: int64

The highest recommendation, item 258, to user 20 is not the item with the highest score. However, it is the highest scoring item that user 20 has never rated.

In [12]:
ratings.loc[(ratings['user'] == 20) & (ratings['item'] == 258)]

,user,item,rating,timestamp


In [13]:
ratings.loc[(ratings['user'] == 31)].set_index('item')

,user,rating,timestamp
item,,,
886,31,2.0,881547877
484,31,5.0,881548030
682,31,2.0,881547834
302,31,4.0,881547719
135,31,4.0,881548030
...,...,...,...
519,31,4.0,881548053
1022,31,5.0,881547814
1019,31,5.0,881548082


## User-User Collaborative Filtering Algorithm

The goal of the user-user cosin approach is so that the model can be updated at each iteration, rather than retraining the entire mode. This will save computational cost and allow for the dynamics experiments to run much quicker

### Testing out the Scikit-learn Cosin Similarity function with Dummy Data

In [14]:
A =  np.array([[0, 1, 0, 0, 1], [0, 0, 1, 1, 1],[1, 1, 0, 1, 0]])
print(A)

[[0 1 0 0 1]
 [0 0 1 1 1]
 [1 1 0 1 0]]


In [15]:
A[0].reshape(1,-1)

array([[0, 1, 0, 0, 1]])

In [16]:
A[1].reshape(1,-1)

array([[0, 0, 1, 1, 1]])

In [17]:
similarities = cosine_similarity(A[0].reshape(1,-1), A[1].reshape(1,-1))
similarities

array([[0.40824829]])

### Testing out the Scikit-learn Cosin Similarity function with ML100K

In [41]:
%%time
user1_ratings = ratings[ratings['user'].values == 99].set_index('item')
user2_ratings = ratings[ratings['user'].values == 100].set_index('item')

CPU times: user 244 µs, sys: 1.9 ms, total: 2.14 ms
Wall time: 1.68 ms


In [42]:
user1_ratings.index.values

array([   4,  268,   79,  111, 1016,  873,  403,  246,  274,   50,  354,
        742,  181,  182,  597,  410,  168,  204,  895,  508,  546,  682,
        124,  421,  741, 1052,  118,  203, 1119,  346,  258,  123,  196,
        232,   11,  963,  685,    7,   56,  363,  237,  369,  676, 1067,
        100,  322,  331,  312,  245,   69,  265,  926,   12,  255,    3,
        871,  288,  827,  402,  107,   66,  762,  628,  328,  413,  694,
        409,  748,  978,  406,  472,  273,  338,  815,   28,   64,  342,
         92,  751,  829,  275,  619,  315,  125,  845,  358,  345,  201,
        121,  475,   22,  300,  471,  405,  456,  238,    1,  595,  117,
        367,  763,   25,  591,  332,  240, 1079, 1132,  975,  173,  210,
        105,  313,  147,  931,  651,  120,  282,  290,  780,  116, 1048,
        310,  172,  326,  678,  276,  789,   98,  294,  544,  174,  329,
       1047,  473,  433,  348])

In [43]:
user2_ratings.index.values

array([ 344,  354,  268,  321,  355,  750,  266,  288,  302,  340,  689,
        905,  289,  691,  316, 1236,  342,  990,  333,  752,  323,  348,
        313,  292, 1238,  879,  300,  328, 1235, 1237,  678,  286,  908,
        690,  874,  880,  349,  310,  347, 1234,  270, 1233,  326,  269,
        258,  900,  886,  294,  272,  881,  895,  892,  887,  885,  346,
        751,  271,  898,  315])

In [93]:
%%timeit
merge = pd.merge(user1_ratings, user2_ratings, how="inner", left_index=True, right_index=True)

967 µs ± 8.01 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [94]:
%%timeit
df2 = user2_ratings[user2_ratings.index.isin(user1_ratings.index)]
user1_ratings.merge(df2, left_index=True, right_index=True)

1.22 ms ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [95]:
%%timeit
user2_ratings.join(user1_ratings, how="inner", lsuffix='rating', rsuffix='rating')

941 µs ± 19.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [98]:
user2_ratings.join(user1_ratings, how="inner", lsuffix='_x', rsuffix='_y')

,user_x,rating_x,timestamp_x,user_y,rating_y,timestamp_y
item,,,,,,
354,100,2.0,891375260,99,2.0,888469332
268,100,3.0,891374982,99,3.0,885678247
288,100,2.0,891374603,99,4.0,885678247
342,100,3.0,891375454,99,1.0,885678348
348,100,3.0,891375630,99,4.0,886518562
313,100,5.0,891374706,99,5.0,885678348
300,100,4.0,891375112,99,4.0,885678397
328,100,4.0,891375212,99,4.0,885678696
678,100,3.0,891375428,99,2.0,885678479


#### In ndarray representation

In [23]:
%%time
similarities = cosine_similarity(np.array(merge['rating_x']).reshape(1,-1), np.array(merge['rating_y']).reshape(1,-1))
similarities

CPU times: user 624 µs, sys: 184 µs, total: 808 µs
Wall time: 797 µs


array([[0.96814626]], dtype=float32)

### Building the CosinSimilarity Class

In [132]:
class CosinSimilarity(Recommender, Predictor):
    """
    Recommend new items by finding users that are the most similar to the given users using the cosin distance formula
    
    Args:
        selector(CandidateSelector):
            The candidate selector to use. If ``None``, uses a new
            :class:`UnratedItemCandidateSelector`.

    """
    def __init__(self, num_users, selector = None):
        
        ## Set selector
        if selector is None:
            self.selector = basic.UnratedItemCandidateSelector()
        else:
            self.selector = selector
        
        ## Set number of users in original matrix
        self.num_users = num_users
        
        ## Instantiate sparse matrix with diagonal
        self.sim_matrix = sparse.eye(num_users)
        
        ## Convert diagonal to lil
        self.sim_matrix = self.sim_matrix.tolil()
        
    ## Input the ratings matrix
    def fit(self, ratings, **kwargs):
        
        # Store each user's rating df in dictionary for quick lookup
        self.df_dict = {}
        for i in range(1, self.num_users+1):
            self.df_dict[i] = ratings[ratings['user'].values == i].set_index('item')
            self.df_dict[i].sort_index(inplace=True)
        
        # Populate sparse matrix for all users with cos sim
        for i in range(1, self.num_users+1):
            for j in range (i+1, self.num_users+1):
                user_x = self.df_dict[i]
                user_y = self.df_dict[j]

                ## METHOD1: using inner join (marginally faster)
                merge = user_y.join(user_x, how="inner", lsuffix='_x', rsuffix='_y')
                    
                ## METHOD2: using inner merge (marginally slower)
                # merge = pd.merge(user_x, user_y, how="inner", left_index=True, right_index=True)

                if (len(merge) > 5):
                    similarity = cosine_similarity(np.array(merge['rating_x']).reshape(1,-1), np.array(merge['rating_y']).reshape(1,-1))
                    self.sim_matrix[i-1, j-1] = similarity
                else:
                    self.sim_matrix[i-1, j-1] = 0
        
        # Reduce candidate space to unseen items
        self.selector.fit(ratings)
    
    ## Provide a recommendation of top "n" movies given "user"
    ## The recommender uses the UnratedItemCandidateSelector by default and uses the ratings matrix 
    ## it was originally fit on
    def recommend(self, user, n=None, candidates=None, ratings=None):
        
        # Obtain reduced candidate space
        if candidates is None:
            candidates = self.selector.candidates(user, ratings)   
        
        # Obtain predictions from reduced candidate space
        predict_for_user(user, candidates, ratings)
        
        pass 
    
    def predict_for_user(self, user, items, ratings=None):
        pass
    
    def __str__(self):
        return 'CosinSimilarity'

In [134]:
%%time
# Instantiate object
num_users = 50 # set to len(users)
algo_cosin = CosinSimilarity(num_users)

# Reduce the candidates space + build user-user cosin similarity matrix 
algo_cosin.fit(ratings)
pp(algo_cosin.sim_matrix.toarray())

array([[1.        , 0.96058202, 0.85707468, ..., 0.93851924, 0.87279063,
        0.92135477],
       [0.        , 1.        , 0.93560153, ..., 0.87504452, 0.91667771,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.87682891,
        0.89637583],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.8188194 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.89708692],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])
CPU times: user 1.47 s, sys: 7 µs, total: 1.47 s
Wall time: 1.47 s


# References
Relevant references:
1. https://realpython.com/build-recommendation-engine-collaborative-filtering/
1. https://lkpy.readthedocs.io/en/stable/GettingStarted.html#
1. https://github.com/lenskit/lkpy/tree/main/lenskit/algorithms
1. https://link.springer.com/book/10.1007%2F978-3-319-29659-3